## Deliverable 2. Create a Customer Travel Destinations Map.

In [38]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import matplotlib.pyplot as plt


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [14]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cotonou,BJ,6.3654,2.4183,80.58,83,40,11.50,scattered clouds
1,1,Pizarro,CO,4.9533,-77.3660,79.27,83,74,7.16,broken clouds
2,2,Atuona,PF,-9.8000,-139.0333,78.31,77,33,21.56,scattered clouds
3,3,Ribeira Grande,PT,38.5167,-28.7000,69.93,68,36,10.47,scattered clouds
4,4,Rikitea,PF,-23.1203,-134.9692,73.96,76,4,11.83,clear sky


In [20]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [21]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cotonou,BJ,6.3654,2.4183,80.58,83,40,11.50,scattered clouds
1,1,Pizarro,CO,4.9533,-77.3660,79.27,83,74,7.16,broken clouds
2,2,Atuona,PF,-9.8000,-139.0333,78.31,77,33,21.56,scattered clouds
3,3,Ribeira Grande,PT,38.5167,-28.7000,69.93,68,36,10.47,scattered clouds
4,4,Rikitea,PF,-23.1203,-134.9692,73.96,76,4,11.83,clear sky


In [23]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [26]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [27]:
preferred_cities_df.count()

City_ID                430
City                   430
Country                430
Lat                    430
Lng                    430
Max Temp               430
Humidity               430
Cloudiness             430
Wind Speed             430
Current Description    430
dtype: int64

In [28]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cotonou,BJ,80.58,scattered clouds,6.3654,2.4183,
1,Pizarro,CO,79.27,broken clouds,4.9533,-77.3660,
2,Atuona,PF,78.31,scattered clouds,-9.8000,-139.0333,
3,Ribeira Grande,PT,69.93,scattered clouds,38.5167,-28.7000,
4,Rikitea,PF,73.96,clear sky,-23.1203,-134.9692,
7,Arraial Do Cabo,BR,73.26,broken clouds,-22.9661,-42.0278,
9,Erzin,TR,75.92,clear sky,36.9559,36.2000,
10,Faanui,PF,77.94,light rain,-16.4833,-151.7500,
11,Hilo,US,81.25,light rain,19.7297,-155.0900,
14,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,


In [32]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        hotel_df['Hotel Name'].replace('', np.nan, inplace=True)

In [34]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cotonou,BJ,80.58,scattered clouds,6.3654,2.4183,Novotel Cotonou Orisha
1,Pizarro,CO,79.27,broken clouds,4.9533,-77.3660,Hotel Limonar
2,Atuona,PF,78.31,scattered clouds,-9.8000,-139.0333,Villa Enata
3,Ribeira Grande,PT,69.93,scattered clouds,38.5167,-28.7000,Quinta da Meia Eira
4,Rikitea,PF,73.96,clear sky,-23.1203,-134.9692,People ThankYou
...,...,...,...,...,...,...,...
703,Embonas,GR,74.84,clear sky,36.2269,27.8564,Hotel Ataviros
704,Emilio Carranza,MX,81.93,broken clouds,19.9711,-96.6114,casa carranza
712,Tautira,PF,82.42,broken clouds,-17.7333,-149.1500,Pension Oaoa
713,San Juan Del Cesar,CO,81.79,overcast clouds,10.7711,-73.0031,Hotel Saraje


In [35]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [36]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [37]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [39]:
plt.savefig('WeatherPy_vacation_map.png')

<Figure size 432x288 with 0 Axes>